# YOLOv4-tiny: DISTANCE DETECTOR

### MODULES

In [1]:
import cv2
import numpy as np
import time
from numba import jit,float32
from scipy.spatial import distance as dist
import os

cv2. __version__


'4.4.0'

### IMPORT ALGORITH

In [2]:
#Func_net = cv2.dnn.readNet( weights_file,cfgfile)
net = cv2.dnn.readNet("yolov4-tiny.weights", "yolov4_tiny.cfg")
classes = []

with open("coco.names","r") as f:
    classes = [line.strip() for line in f.readlines()]

layer_names = net.getLayerNames()
output_layers = [layer_names[i[0]-1] for i in net.getUnconnectedOutLayers()]        
colors = np.random.uniform(0,255, size=(len(classes),3))
font =cv2.FONT_HERSHEY_PLAIN


In [3]:
@jit(nopython = True) # +3 FPS   
def PERSON_FLTR_1(OUTS, THRESHOLD, H,W):
    
    class_ids = []
    confidences = []
    boxes = []
    centroids = []
    
    for out in OUTS:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            
            
            # PERSON DETECTED
            if (confidence > THRESHOLD) and class_id == 0: 

                center_x = int(detection[0]*W) 
                #from the bounding box to the center
                center_y = int(detection[1]*H) 
                #from the bb to cy
                w = int(detection[2]*W)
                h = int(detection[3]*H)
                
                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
                centroids.append((center_x,center_y))
                
                
    return   class_ids,confidences, boxes, centroids 


def PERSON_FLTR_2(IDX,ID,CONF, BB, CENTROIDS):
    
    results = []
    # loop over the indexes we are keeping
    for i in IDX.flatten():
        # extract the bounding box coordinates
        (x, y) = (BB[i][0], BB[i][1])
        (w, h) = (BB[i][2], BB[i][3])
        # update our results list to consist of the person
        # prediction probability, bounding box coordinates,
        # and the centroid
        r = (CONF[i], (x, y, x + w, y + h), CENTROIDS[i])
        results.append(r)
    # return the list of results
    return results
    

# START CAPTURING

In [5]:
# define the minimum safe distance (in pixels) that two people can be
# from each other
MIN_DISTANCE = 50
n = 0 #camera
THRESHOLD = 0.4
H, W = 480,640
cap = cv2.VideoCapture(n)#0 for the first webcam and 1 for the second one
                        #...and if we want just put a video folder here
#GET FRAMES IN REAL TIME:
start_time = time.time() #colect start time
frame_id = 0 #colects frame quanty


while True:
    _,frame = cap.read()
    frame_id += 1
    # PARA SABER RESOLUCIÓN JETSON  = > height, width, _ = frame.shape
    
    #Standard Yolo Size 320x 320 /416 x 416
    # Function = blobFromImage(imag, scale_factor, size,mean*, swapRB*, crop) 
    blob = cv2.dnn.blobFromImage(frame, (1 / 255.0),
                                 (320,320),(0,0,0), True, crop = False)
    
    net.setInput(blob)
    # 2 Detections (300, 85) / (1200, 85)    
    OUTS = net.forward(output_layers)
    
    class_ids,confidences, boxes, centroids = PERSON_FLTR_1(OUTS, THRESHOLD, H,W)

                
    #NON MAX SUPRESSION
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.6,0.5)
    # initialize the set of indexes that violate the minimum social
    # distance
    violate = set()

    # if not this...error
    if len(indexes) > 0:
        results = PERSON_FLTR_2(indexes,class_ids,confidences, boxes, centroids)
        
        if len(results) >= 2:
            # extract all centroids from the results and compute the
            # Euclidean distances between all pairs of the centroids
            centroids = np.array([r[2] for r in results])
            D = dist.cdist(centroids, centroids, metric="euclidean")
            # loop over the upper triangular of the distance matrix
            for i in range(0, D.shape[0]):
                for j in range(i + 1, D.shape[1]):
                    # check to see if the distance between any two
                    # centroid pairs is less than the configured number
                    # of pixels
                    if D[i, j] < MIN_DISTANCE:
                        # UPDATE VIOLATIONS
                        violate.add(i)
                        violate.add(j)
                        # FLAG TO SAVE IMAGE

        # loop over the results
        for (i, (prob, bbox, centroid)) in enumerate(results):
            # extract the bounding box and centroid coordinates, then
            # initialize the color of the annotation
            (startX, startY, endX, endY) = bbox
            (cX, cY) = centroid
            color = (0, 255, 0)
            # if the index pair exists within the violation set, then
            # update the color
            if i in violate:
                color = (0, 0, 255)
            # draw (1) a bounding box around the person and (2) the
            # centroid coordinates of the person,
            cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
            cv2.circle(frame, (cX, cY), 5, color, 1)
        # draw the total number of social distancing violations on the
        # output frame
        text = "Social Distancing Violations: {}".format(len(violate))
        cv2.putText(frame, text, (10, frame.shape[0] - 25),
            cv2.FONT_HERSHEY_SIMPLEX, 0.85, (0, 0, 255), 3)


    #SPEED TESTER
    elapse = time.time()- start_time  
    
    fps = frame_id/elapse
    #function_putText = (img, text, org, fontFace, fontScale, color, thikness, lineType, botonLeft)
    cv2.putText( frame, "FPS: " + str(round(fps,2)),(10,30), font, 3,(0,255,0),1)
    
    
    
    #OUTPUT IMAGE
    cv2.imshow("Scratch_YOLO_CV_2.1", frame)
    key = cv2.waitKey(1)
    
    
    
    #STOP COMMANDS
    if key == 27: #esc keyboard
        break
    
cap.release() #Close camera reading
cv2.destroyAllWindows()

0.6366534233093262
0.7642261981964111
0.8345882892608643
0.9027109146118164
0.9634137153625488
1.0349750518798828
1.1007976531982422
1.1723196506500244
1.2311797142028809
1.3025774955749512
1.3695573806762695
1.440032958984375
1.5054197311401367
1.5709147453308105
1.6397573947906494
1.7126426696777344
1.772547721862793
1.8349709510803223
1.8986423015594482
1.9704890251159668
2.0298893451690674
2.0978610515594482
2.1648240089416504
2.2285592555999756
2.2971279621124268
2.3684213161468506
2.437272787094116
2.5082080364227295
2.5716392993927
2.6384928226470947
2.7114460468292236
2.7701523303985596
2.8310766220092773
2.9049367904663086
2.9724130630493164
3.030703067779541
3.0979151725769043
3.164731502532959
3.228130578994751
3.2976770401000977
3.3670802116394043
3.431868553161621
3.499211549758911
3.5647830963134766
3.6326465606689453
3.69985032081604
3.7649166584014893
3.827991247177124
3.896554470062256
3.966067314147949
4.034693241119385
4.098297595977783
4.167258024215698
4.2320060729

30.303491353988647
30.376636028289795
30.440227031707764
30.50378704071045
30.572123050689697
30.639272212982178
30.700480222702026
30.769343614578247
30.83741044998169
30.90097951889038
30.970465660095215
31.04675006866455
31.10590362548828
31.173535585403442
31.2410786151886
31.302687644958496
31.371635675430298
31.43690276145935
31.570589065551758
31.703782320022583
31.841838598251343
31.97751784324646
32.101988554000854
32.24269986152649
32.373889684677124
32.501795053482056
32.6405565738678
32.77143359184265
32.90976023674011
33.04607701301575
33.173871755599976
33.305277585983276
33.43904662132263
33.589699029922485
33.70985651016235
33.839656352996826
33.971532106399536
34.10939168930054
34.24083495140076
34.37025690078735
34.50142049789429
34.639931201934814
34.77551317214966
34.9081757068634
35.03755712509155
35.17571759223938
35.3013858795166
35.43688893318176
35.57689428329468
35.7084686756134
35.83854866027832
35.97557616233826
36.10418343544006
36.24124336242676
36.3715429